## 12. Feature Selection

**A New Enron Feature Quiz**

The ```reader.py``` and ```poi_emails.py``` are not available for editing as per Udacity and I could not find online elsewhere at time of writing this., so I am just sharing the answer part below for quiz

```python
    if from_emails[0] in poi_email_list: # only one "from" possible for any mail
        from_poi = True
```

A mail can have many ```To``` email addresses, and ```cc``` email addresses, but only one ```From``` address. So in our module, for given mail, we just check if From address matches with any of ```POI``` addresses, and set the flag to true if its so.

```from_emails[0]``` because its a single element list.

**Visualizing Your New Feature**

Here we will calculate fraction of no of messages to/from poi to a person to total no of messages to/from a person.

Below is the working snippet. fraction calculated only when possible. NaN and divide by zero are avoided. 

```python
    if poi_messages == 'NaN' or all_messages == 'NaN':
        return 0
        
    if poi_messages != 'NaN' and all_messages != 'NaN':
        if all_messages != 0:
            fraction = poi_messages/all_messages
        else:
            fraction = 0

    return fraction
```

If python 2, do not forget to use ```from __future__ import division``` at the beginning of the script.

**Lasso Prediction**

Code only. Quizzes were simple and straight forward to answer easily.
```python
from sklearn import linear_model
features, labels = GetMyData()
regression = Lasso()
regression.fit(features, labels)
regression.predict([[2,4]])  # not list of lists
print regression.coef_
```

## Mini Project

It starts talking about problem: Katie trying to overfit decision tree with less training set, and/or lots of features. 

An overfit decision tree will have **high** accuracy on training set, but **low** accuracy on testing set.

**Get a decision tree up and running** on training data and print out the accuracy. 

How many training points are there, according to the starter code?

_Note, the pkl file paths changed to suit my folder org accordingly_

In [4]:
#find_signature.py

#!/usr/bin/python

import pickle
import numpy
numpy.random.seed(42)


### The words (features) and authors (labels), already largely processed.
### These files should have been created from the previous (Lesson 10)
### mini-project.
words_file = "../11. Text Learning/your_word_data.pkl"        # CHANGED TO SUIT MY FOLDERS
authors_file = "../11. Text Learning/your_email_authors.pkl"  # CHANGED TO SUIT MY FOLDERS
word_data = pickle.load( open(words_file, "r"))
authors = pickle.load( open(authors_file, "r") )


### test_size is the percentage of events assigned to the test set (the
### remainder go into training)
### feature matrices changed to dense representations for compatibility with
### classifier functions in versions 0.15.2 and earlier
from sklearn import cross_validation
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')
features_train = vectorizer.fit_transform(features_train)
features_test  = vectorizer.transform(features_test).toarray()


### a classic way to overfit is to use a small number
### of data points and a large number of features;
### train on only 150 events to put ourselves in this regime
features_train = features_train[:150].toarray()
labels_train   = labels_train[:150]


### your code goes here
from sklearn import tree
from sklearn.metrics import accuracy_score
clf = tree.DecisionTreeClassifier()
clf = clf.fit(features_train,labels_train)
labels_pred = clf.predict(features_test)
acc = accuracy_score(labels_test, labels_pred)
print acc

0.9476678043230944


The length of training data is 150 as given in comments in above code. 

**Identify the Most Powerful Features**  

Take your (overfit) decision tree and use the ```feature_importances_``` attribute to get a list of the relative importance of all the features being used. We suggest iterating through this list (it’s long, since this is text data) and only printing out the feature importance if it’s above some threshold (say, 0.2--remember, if all words were equally important, each one would give an importance of far less than 0.01). What’s the importance of the most important feature? What is the number of this feature?

In [12]:
# ok, what is the length of feature importances list
feature_imp_list = clf.feature_importances_
len(feature_imp_list)

37863

In [17]:
# ok quite high, so printing only if above threshold..
import numpy as np
for i,val in np.ndenumerate(feature_imp_list):
    if val > 0.2:
        print i,val

(33614L,) 0.7647058823529412


**Use TfIdf to Get the Most Important Word**  

In order to figure out what words are causing the problem, you need to go back to the TfIdf and use the feature numbers that you obtained in the previous part of the mini-project to get the associated words. You can return a list of all the words in the TfIdf by calling get_feature_names() on it; pull out the word that’s causing most of the discrimination of the decision tree. What is it? Does it make sense as a word that’s uniquely tied to either Chris Germany or Sara Shackleton, a signature of sorts?

In [19]:
vectorizer.get_feature_names()[33614]

u'sshacklensf'

**Remove, Repeat**

This word seems like an outlier in a certain sense, so let’s remove it and refit. Go back to ```text_learning/vectorize_text.py```, and remove this word from the emails using the same method you used to remove “sara”, “chris”, etc. Rerun ```vectorize_text.py```, and once that finishes, rerun find_signature.py. Any other outliers pop up? What word is it? Seem like a signature-type word? (Define an outlier as a feature with importance >0.2, as before).  

<br><font color=blue>
**Note:**
Since we dont want to touch existing code, and to illustrate bigger picture, we will use ```vectorize_code.py``` and its dependencies here as well, so hold on, lengthy coding ahead</font>

```vectorize_code.py:``` changes indicated by ```# CHANGE```

In [22]:
#vectorize_text.py

#!/usr/bin/python
import os
import pickle
import re
import sys

sys.path.append( "../tools/" )
from parse_out_email_text import parseOutText

"""
    Starter code to process the emails from Sara and Chris to extract
    the features and get the documents ready for classification.

    The list of all the emails from Sara are in the from_sara list
    likewise for emails from Chris (from_chris)

    The actual documents are in the Enron email dataset, which
    you downloaded/unpacked in Part 0 of the first mini-project. If you have
    not obtained the Enron email corpus, run startup.py in the tools folder.

    The data is stored in lists and packed away in pickle files at the end.
"""


from_sara  = open("../11. Text Learning/from_sara.txt", "r")    # CHANGE
from_chris = open("../11. Text Learning/from_chris.txt", "r")   # CHANGE

from_data = []
word_data = []

### temp_counter is a way to speed up the development--there are
### thousands of emails from Sara and Chris, so running over all of them
### can take a long time
### temp_counter helps you only look at the first 200 emails in the list so you
### can iterate your modifications quicker
temp_counter = 0


for name, from_person in [("sara", from_sara), ("chris", from_chris)]:
    for path in from_person:
        ### only look at first 200 emails when developing
        ### once everything is working, remove this line to run over full dataset
        #temp_counter += 1
        #if temp_counter < 200:
            path = os.path.join('..', path[:-1])
            print path
            email = open(path, "r")

            ### use parseOutText to extract the text from the opened email
            stemmed_email = parseOutText(email)        

            ### use str.replace() to remove any instances of the words
            ### ["sara", "shackleton", "chris", "germani"]
            signature_words = ["sara", "shackleton", "chris", "germani", "sshacklensf"] # CHANGE
            for each_word in signature_words:
                stemmed_email = stemmed_email.replace(each_word,'')

            ### append the text to word_data
            word_data.append(stemmed_email)

            ### append a 0 to from_data if email is from Sara, and 1 if email is from Chris
            if name == "sara": 
                from_data.append(0)
            else: # its chris
                from_data.append(1)


            email.close()

print "emails processed"
from_sara.close()
from_chris.close()

pickle.dump( word_data, open("your_word_data.pkl", "w") )
pickle.dump( from_data, open("your_email_authors.pkl", "w") )

print 'Done'
### in Part 4, do TfIdf vectorization here

..\maildir/bailey-s/deleted_items/101.
..\maildir/bailey-s/deleted_items/106.
..\maildir/bailey-s/deleted_items/132.
..\maildir/bailey-s/deleted_items/185.
..\maildir/bailey-s/deleted_items/186.
..\maildir/bailey-s/deleted_items/187.
..\maildir/bailey-s/deleted_items/193.
..\maildir/bailey-s/deleted_items/195.
..\maildir/bailey-s/deleted_items/214.
..\maildir/bailey-s/deleted_items/215.
..\maildir/bailey-s/deleted_items/233.
..\maildir/bailey-s/deleted_items/242.
..\maildir/bailey-s/deleted_items/243.
..\maildir/bailey-s/deleted_items/244.
..\maildir/bailey-s/deleted_items/246.
..\maildir/bailey-s/deleted_items/247.
..\maildir/bailey-s/deleted_items/254.
..\maildir/bailey-s/deleted_items/259.
..\maildir/bailey-s/deleted_items/260.
..\maildir/bailey-s/deleted_items/261.
..\maildir/bailey-s/deleted_items/263.
..\maildir/bailey-s/deleted_items/278.
..\maildir/bailey-s/deleted_items/290.
..\maildir/bailey-s/deleted_items/296.
..\maildir/bailey-s/deleted_items/302.
..\maildir/bailey-s/delet

..\maildir/jones-t/notes_inbox/1741.
..\maildir/jones-t/notes_inbox/1750.
..\maildir/jones-t/notes_inbox/1760.
..\maildir/jones-t/notes_inbox/1763.
..\maildir/jones-t/notes_inbox/1796.
..\maildir/jones-t/notes_inbox/1804.
..\maildir/jones-t/notes_inbox/1851.
..\maildir/jones-t/notes_inbox/1863.
..\maildir/jones-t/notes_inbox/189.
..\maildir/jones-t/notes_inbox/1898.
..\maildir/jones-t/notes_inbox/1918.
..\maildir/jones-t/notes_inbox/1923.
..\maildir/jones-t/notes_inbox/1925.
..\maildir/jones-t/notes_inbox/1929.
..\maildir/jones-t/notes_inbox/208.
..\maildir/jones-t/notes_inbox/2091.
..\maildir/jones-t/notes_inbox/2177.
..\maildir/jones-t/notes_inbox/2180.
..\maildir/jones-t/notes_inbox/2214.
..\maildir/jones-t/notes_inbox/2234.
..\maildir/jones-t/notes_inbox/2235.
..\maildir/jones-t/notes_inbox/2244.
..\maildir/jones-t/notes_inbox/2285.
..\maildir/jones-t/notes_inbox/2335.
..\maildir/jones-t/notes_inbox/2361.
..\maildir/jones-t/notes_inbox/2374.
..\maildir/jones-t/notes_inbox/2414.
..\

..\maildir/shackleton-s/all_documents/10354.
..\maildir/shackleton-s/all_documents/10355.
..\maildir/shackleton-s/all_documents/10359.
..\maildir/shackleton-s/all_documents/1036.
..\maildir/shackleton-s/all_documents/10361.
..\maildir/shackleton-s/all_documents/1037.
..\maildir/shackleton-s/all_documents/10372.
..\maildir/shackleton-s/all_documents/10373.
..\maildir/shackleton-s/all_documents/10374.
..\maildir/shackleton-s/all_documents/1038.
..\maildir/shackleton-s/all_documents/10386.
..\maildir/shackleton-s/all_documents/10387.
..\maildir/shackleton-s/all_documents/10389.
..\maildir/shackleton-s/all_documents/1039.
..\maildir/shackleton-s/all_documents/104.
..\maildir/shackleton-s/all_documents/1040.
..\maildir/shackleton-s/all_documents/10403.
..\maildir/shackleton-s/all_documents/10405.
..\maildir/shackleton-s/all_documents/1041.
..\maildir/shackleton-s/all_documents/10412.
..\maildir/shackleton-s/all_documents/10418.
..\maildir/shackleton-s/all_documents/1042.
..\maildir/shacklet

..\maildir/shackleton-s/all_documents/10755.
..\maildir/shackleton-s/all_documents/10756.
..\maildir/shackleton-s/all_documents/10758.
..\maildir/shackleton-s/all_documents/1076.
..\maildir/shackleton-s/all_documents/10761.
..\maildir/shackleton-s/all_documents/10762.
..\maildir/shackleton-s/all_documents/10765.
..\maildir/shackleton-s/all_documents/1077.
..\maildir/shackleton-s/all_documents/10776.
..\maildir/shackleton-s/all_documents/10777.
..\maildir/shackleton-s/all_documents/1078.
..\maildir/shackleton-s/all_documents/10788.
..\maildir/shackleton-s/all_documents/1079.
..\maildir/shackleton-s/all_documents/10793.
..\maildir/shackleton-s/all_documents/10798.
..\maildir/shackleton-s/all_documents/10799.
..\maildir/shackleton-s/all_documents/108.
..\maildir/shackleton-s/all_documents/1080.
..\maildir/shackleton-s/all_documents/10802.
..\maildir/shackleton-s/all_documents/10803.
..\maildir/shackleton-s/all_documents/10804.
..\maildir/shackleton-s/all_documents/10807.
..\maildir/shackl

..\maildir/shackleton-s/all_documents/11264.
..\maildir/shackleton-s/all_documents/11265.
..\maildir/shackleton-s/all_documents/11266.
..\maildir/shackleton-s/all_documents/1127.
..\maildir/shackleton-s/all_documents/11270.
..\maildir/shackleton-s/all_documents/11271.
..\maildir/shackleton-s/all_documents/11273.
..\maildir/shackleton-s/all_documents/11274.
..\maildir/shackleton-s/all_documents/11277.
..\maildir/shackleton-s/all_documents/11279.
..\maildir/shackleton-s/all_documents/11281.
..\maildir/shackleton-s/all_documents/11283.
..\maildir/shackleton-s/all_documents/11291.
..\maildir/shackleton-s/all_documents/11292.
..\maildir/shackleton-s/all_documents/11294.
..\maildir/shackleton-s/all_documents/11296.
..\maildir/shackleton-s/all_documents/11297.
..\maildir/shackleton-s/all_documents/11299.
..\maildir/shackleton-s/all_documents/113.
..\maildir/shackleton-s/all_documents/1130.
..\maildir/shackleton-s/all_documents/11302.
..\maildir/shackleton-s/all_documents/11308.
..\maildir/sha

...

..\maildir/germany-c/sent_items/213.
..\maildir/germany-c/sent_items/214.
..\maildir/germany-c/sent_items/215.
..\maildir/germany-c/sent_items/216.
..\maildir/germany-c/sent_items/217.
..\maildir/germany-c/sent_items/218.
..\maildir/germany-c/sent_items/219.
..\maildir/germany-c/sent_items/22.
..\maildir/germany-c/sent_items/220.
..\maildir/germany-c/sent_items/221.
..\maildir/germany-c/sent_items/222.
..\maildir/germany-c/sent_items/223.
..\maildir/germany-c/sent_items/224.
..\maildir/germany-c/sent_items/225.
..\maildir/germany-c/sent_items/226.
..\maildir/germany-c/sent_items/227.
..\maildir/germany-c/sent_items/228.
..\maildir/germany-c/sent_items/229.
..\maildir/germany-c/sent_items/23.
..\maildir/germany-c/sent_items/230.
..\maildir/germany-c/sent_items/231.
..\maildir/germany-c/sent_items/232.
..\maildir/germany-c/sent_items/234.
..\maildir/germany-c/sent_items/235.
..\maildir/germany-c/sent_items/236.
..\maildir/germany-c/sent_items/237.
..\maildir/germany-c/sent_items/238.
..\

..\maildir/germany-c/sent_items/438.
..\maildir/germany-c/sent_items/439.
..\maildir/germany-c/sent_items/44.
..\maildir/germany-c/sent_items/440.
..\maildir/germany-c/sent_items/441.
..\maildir/germany-c/sent_items/442.
..\maildir/germany-c/sent_items/443.
..\maildir/germany-c/sent_items/444.
..\maildir/germany-c/sent_items/445.
..\maildir/germany-c/sent_items/446.
..\maildir/germany-c/sent_items/447.
..\maildir/germany-c/sent_items/448.
..\maildir/germany-c/sent_items/449.
..\maildir/germany-c/sent_items/45.
..\maildir/germany-c/sent_items/450.
..\maildir/germany-c/sent_items/451.
..\maildir/germany-c/sent_items/452.
..\maildir/germany-c/sent_items/453.
..\maildir/germany-c/sent_items/454.
..\maildir/germany-c/sent_items/455.
..\maildir/germany-c/sent_items/456.
..\maildir/germany-c/sent_items/457.
..\maildir/germany-c/sent_items/458.
..\maildir/germany-c/sent_items/459.
..\maildir/germany-c/sent_items/46.
..\maildir/germany-c/sent_items/460.
..\maildir/germany-c/sent_items/461.
..\m

..\maildir/germany-c/sent_items/656.
..\maildir/germany-c/sent_items/657.
..\maildir/germany-c/sent_items/658.
..\maildir/germany-c/sent_items/659.
..\maildir/germany-c/sent_items/66.
..\maildir/germany-c/sent_items/660.
..\maildir/germany-c/sent_items/661.
..\maildir/germany-c/sent_items/662.
..\maildir/germany-c/sent_items/663.
..\maildir/germany-c/sent_items/664.
..\maildir/germany-c/sent_items/665.
..\maildir/germany-c/sent_items/666.
..\maildir/germany-c/sent_items/667.
..\maildir/germany-c/sent_items/668.
..\maildir/germany-c/sent_items/669.
..\maildir/germany-c/sent_items/67.
..\maildir/germany-c/sent_items/670.
..\maildir/germany-c/sent_items/671.
..\maildir/germany-c/sent_items/672.
..\maildir/germany-c/sent_items/673.
..\maildir/germany-c/sent_items/674.
..\maildir/germany-c/sent_items/675.
..\maildir/germany-c/sent_items/676.
..\maildir/germany-c/sent_items/677.
..\maildir/germany-c/sent_items/678.
..\maildir/germany-c/sent_items/679.
..\maildir/germany-c/sent_items/68.
..\m

..\maildir/germany-c/sent_items/86.
..\maildir/germany-c/sent_items/860.
..\maildir/germany-c/sent_items/861.
..\maildir/germany-c/sent_items/862.
..\maildir/germany-c/sent_items/863.
..\maildir/germany-c/sent_items/864.
..\maildir/germany-c/sent_items/865.
..\maildir/germany-c/sent_items/866.
..\maildir/germany-c/sent_items/867.
..\maildir/germany-c/sent_items/868.
..\maildir/germany-c/sent_items/869.
..\maildir/germany-c/sent_items/87.
..\maildir/germany-c/sent_items/870.
..\maildir/germany-c/sent_items/871.
..\maildir/germany-c/sent_items/872.
..\maildir/germany-c/sent_items/873.
..\maildir/germany-c/sent_items/874.
..\maildir/germany-c/sent_items/875.
..\maildir/germany-c/sent_items/876.
..\maildir/germany-c/sent_items/877.
..\maildir/germany-c/sent_items/878.
..\maildir/germany-c/sent_items/879.
..\maildir/germany-c/sent_items/88.
..\maildir/germany-c/sent_items/880.
..\maildir/germany-c/sent_items/881.
..\maildir/germany-c/sent_items/882.
..\maildir/germany-c/sent_items/883.
..\m

..\maildir/parks-j/deleted_items/449.
..\maildir/parks-j/deleted_items/463.
..\maildir/parks-j/deleted_items/474.
..\maildir/parks-j/deleted_items/483.
..\maildir/parks-j/deleted_items/505.
..\maildir/parks-j/deleted_items/520.
..\maildir/parks-j/deleted_items/526.
..\maildir/parks-j/deleted_items/528.
..\maildir/parks-j/deleted_items/546.
..\maildir/parks-j/deleted_items/578.
..\maildir/parks-j/deleted_items/580.
..\maildir/parks-j/deleted_items/581.
..\maildir/parks-j/deleted_items/600.
..\maildir/parks-j/deleted_items/601.
..\maildir/parks-j/deleted_items/795.
..\maildir/parks-j/deleted_items/821.
..\maildir/parks-j/deleted_items/824.
..\maildir/parks-j/deleted_items/828.
..\maildir/parks-j/inbox/626.
..\maildir/parks-j/inbox/635.
..\maildir/parks-j/inbox/636.
..\maildir/parks-j/inbox/638.
..\maildir/parks-j/inbox/655.
..\maildir/parks-j/inbox/665.
..\maildir/parks-j/inbox/685.
..\maildir/parks-j/inbox/728.
..\maildir/parks-j/inbox/732.
..\maildir/parks-j/inbox/733.
..\maildir/parks

Now that I vectorize here, the output pickle files are produced in current folder (so some day any one looking for reference would get difference between earlier pickle files and after these updates)  

Ok, let us rerun ```find_signature.py``` again and check (remember to change pickle path to read new pickle files just generated), as usual change indicated by ```# CHANGE```

In [23]:
#find_signature.py

#!/usr/bin/python

import pickle
import numpy
numpy.random.seed(42)


### The words (features) and authors (labels), already largely processed.
### These files should have been created from the previous (Lesson 10)
### mini-project.
words_file = "your_word_data.pkl"        # CHANGE
authors_file = "your_email_authors.pkl"  # CHANGE
word_data = pickle.load( open(words_file, "r"))
authors = pickle.load( open(authors_file, "r") )


### test_size is the percentage of events assigned to the test set (the
### remainder go into training)
### feature matrices changed to dense representations for compatibility with
### classifier functions in versions 0.15.2 and earlier
from sklearn import cross_validation
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')
features_train = vectorizer.fit_transform(features_train)
features_test  = vectorizer.transform(features_test).toarray()


### a classic way to overfit is to use a small number
### of data points and a large number of features;
### train on only 150 events to put ourselves in this regime
features_train = features_train[:150].toarray()
labels_train   = labels_train[:150]


### your code goes here
from sklearn import tree
from sklearn.metrics import accuracy_score
clf = tree.DecisionTreeClassifier()
clf = clf.fit(features_train,labels_train)
labels_pred = clf.predict(features_test)
acc = accuracy_score(labels_test, labels_pred)
print acc

0.9505119453924915


In [24]:
# ok, what is the length of feature importances list
feature_imp_list = clf.feature_importances_
len(feature_imp_list)

37862

In [25]:
# ok quite high, so printing only if above threshold..
import numpy as np
for i,val in np.ndenumerate(feature_imp_list):
    if val > 0.2:
        print i,val

(14343L,) 0.6666666666666667


Looks like another outlier popping up.. 

In [26]:
vectorizer.get_feature_names()[14343]

u'cgermannsf'

**Checking Important Features Again**

Lets remove ```cgermannsf```, rinse and repeat.. 

In [27]:
#vectorize_text.py

#!/usr/bin/python
import os
import pickle
import re
import sys

sys.path.append( "../tools/" )
from parse_out_email_text import parseOutText

"""
    Starter code to process the emails from Sara and Chris to extract
    the features and get the documents ready for classification.

    The list of all the emails from Sara are in the from_sara list
    likewise for emails from Chris (from_chris)

    The actual documents are in the Enron email dataset, which
    you downloaded/unpacked in Part 0 of the first mini-project. If you have
    not obtained the Enron email corpus, run startup.py in the tools folder.

    The data is stored in lists and packed away in pickle files at the end.
"""


from_sara  = open("../11. Text Learning/from_sara.txt", "r")    # CHANGE
from_chris = open("../11. Text Learning/from_chris.txt", "r")   # CHANGE

from_data = []
word_data = []

### temp_counter is a way to speed up the development--there are
### thousands of emails from Sara and Chris, so running over all of them
### can take a long time
### temp_counter helps you only look at the first 200 emails in the list so you
### can iterate your modifications quicker
temp_counter = 0


for name, from_person in [("sara", from_sara), ("chris", from_chris)]:
    for path in from_person:
        ### only look at first 200 emails when developing
        ### once everything is working, remove this line to run over full dataset
        #temp_counter += 1
        #if temp_counter < 200:
            path = os.path.join('..', path[:-1])
            print path
            email = open(path, "r")

            ### use parseOutText to extract the text from the opened email
            stemmed_email = parseOutText(email)        

            ### use str.replace() to remove any instances of the words
            ### ["sara", "shackleton", "chris", "germani"]
            signature_words = ["sara", "shackleton", "chris", "germani", "sshacklensf","cgermannsf"] # CHANGE
            for each_word in signature_words:
                stemmed_email = stemmed_email.replace(each_word,'')

            ### append the text to word_data
            word_data.append(stemmed_email)

            ### append a 0 to from_data if email is from Sara, and 1 if email is from Chris
            if name == "sara": 
                from_data.append(0)
            else: # its chris
                from_data.append(1)


            email.close()

print "emails processed"
from_sara.close()
from_chris.close()

pickle.dump( word_data, open("your_word_data.pkl", "w") )
pickle.dump( from_data, open("your_email_authors.pkl", "w") )

print 'Done'
### in Part 4, do TfIdf vectorization here

..\maildir/bailey-s/deleted_items/101.
..\maildir/bailey-s/deleted_items/106.
..\maildir/bailey-s/deleted_items/132.
..\maildir/bailey-s/deleted_items/185.
..\maildir/bailey-s/deleted_items/186.
..\maildir/bailey-s/deleted_items/187.
..\maildir/bailey-s/deleted_items/193.
..\maildir/bailey-s/deleted_items/195.
..\maildir/bailey-s/deleted_items/214.
..\maildir/bailey-s/deleted_items/215.
..\maildir/bailey-s/deleted_items/233.
..\maildir/bailey-s/deleted_items/242.
..\maildir/bailey-s/deleted_items/243.
..\maildir/bailey-s/deleted_items/244.
..\maildir/bailey-s/deleted_items/246.
..\maildir/bailey-s/deleted_items/247.
..\maildir/bailey-s/deleted_items/254.
..\maildir/bailey-s/deleted_items/259.
..\maildir/bailey-s/deleted_items/260.
..\maildir/bailey-s/deleted_items/261.
..\maildir/bailey-s/deleted_items/263.
..\maildir/bailey-s/deleted_items/278.
..\maildir/bailey-s/deleted_items/290.
..\maildir/bailey-s/deleted_items/296.
..\maildir/bailey-s/deleted_items/302.
..\maildir/bailey-s/delet

..\maildir/jones-t/notes_inbox/2177.
..\maildir/jones-t/notes_inbox/2180.
..\maildir/jones-t/notes_inbox/2214.
..\maildir/jones-t/notes_inbox/2234.
..\maildir/jones-t/notes_inbox/2235.
..\maildir/jones-t/notes_inbox/2244.
..\maildir/jones-t/notes_inbox/2285.
..\maildir/jones-t/notes_inbox/2335.
..\maildir/jones-t/notes_inbox/2361.
..\maildir/jones-t/notes_inbox/2374.
..\maildir/jones-t/notes_inbox/2414.
..\maildir/jones-t/notes_inbox/2431.
..\maildir/jones-t/notes_inbox/2433.
..\maildir/jones-t/notes_inbox/247.
..\maildir/jones-t/notes_inbox/251.
..\maildir/jones-t/notes_inbox/2632.
..\maildir/jones-t/notes_inbox/2701.
..\maildir/jones-t/notes_inbox/2705.
..\maildir/jones-t/notes_inbox/2720.
..\maildir/jones-t/notes_inbox/2730.
..\maildir/jones-t/notes_inbox/2740.
..\maildir/jones-t/notes_inbox/2741.
..\maildir/jones-t/notes_inbox/2770.
..\maildir/jones-t/notes_inbox/2777.
..\maildir/jones-t/notes_inbox/2805.
..\maildir/jones-t/notes_inbox/2806.
..\maildir/jones-t/notes_inbox/283.
..\m

..\maildir/shackleton-s/all_documents/10412.
..\maildir/shackleton-s/all_documents/10418.
..\maildir/shackleton-s/all_documents/1042.
..\maildir/shackleton-s/all_documents/10420.
..\maildir/shackleton-s/all_documents/10421.
..\maildir/shackleton-s/all_documents/10423.
..\maildir/shackleton-s/all_documents/10427.
..\maildir/shackleton-s/all_documents/10429.
..\maildir/shackleton-s/all_documents/1043.
..\maildir/shackleton-s/all_documents/10430.
..\maildir/shackleton-s/all_documents/10431.
..\maildir/shackleton-s/all_documents/10432.
..\maildir/shackleton-s/all_documents/10433.
..\maildir/shackleton-s/all_documents/10435.
..\maildir/shackleton-s/all_documents/10437.
..\maildir/shackleton-s/all_documents/10438.
..\maildir/shackleton-s/all_documents/10439.
..\maildir/shackleton-s/all_documents/1044.
..\maildir/shackleton-s/all_documents/10440.
..\maildir/shackleton-s/all_documents/10443.
..\maildir/shackleton-s/all_documents/10444.
..\maildir/shackleton-s/all_documents/10447.
..\maildir/sh

..\maildir/shackleton-s/all_documents/10830.
..\maildir/shackleton-s/all_documents/10831.
..\maildir/shackleton-s/all_documents/10832.
..\maildir/shackleton-s/all_documents/10839.
..\maildir/shackleton-s/all_documents/1084.
..\maildir/shackleton-s/all_documents/10845.
..\maildir/shackleton-s/all_documents/10846.
..\maildir/shackleton-s/all_documents/10849.
..\maildir/shackleton-s/all_documents/1085.
..\maildir/shackleton-s/all_documents/10852.
..\maildir/shackleton-s/all_documents/1086.
..\maildir/shackleton-s/all_documents/10860.
..\maildir/shackleton-s/all_documents/10862.
..\maildir/shackleton-s/all_documents/10864.
..\maildir/shackleton-s/all_documents/10865.
..\maildir/shackleton-s/all_documents/10867.
..\maildir/shackleton-s/all_documents/1087.
..\maildir/shackleton-s/all_documents/1088.
..\maildir/shackleton-s/all_documents/10882.
..\maildir/shackleton-s/all_documents/10886.
..\maildir/shackleton-s/all_documents/10887.
..\maildir/shackleton-s/all_documents/1089.
..\maildir/shack

..\maildir/shackleton-s/all_documents/11349.
..\maildir/shackleton-s/all_documents/1135.
..\maildir/shackleton-s/all_documents/11350.
..\maildir/shackleton-s/all_documents/11363.
..\maildir/shackleton-s/all_documents/11364.
..\maildir/shackleton-s/all_documents/1137.
..\maildir/shackleton-s/all_documents/11372.
..\maildir/shackleton-s/all_documents/11374.
..\maildir/shackleton-s/all_documents/1138.
..\maildir/shackleton-s/all_documents/11383.
..\maildir/shackleton-s/all_documents/11388.
..\maildir/shackleton-s/all_documents/1139.
..\maildir/shackleton-s/all_documents/11392.
..\maildir/shackleton-s/all_documents/11393.
..\maildir/shackleton-s/all_documents/11394.
..\maildir/shackleton-s/all_documents/11395.
..\maildir/shackleton-s/all_documents/11396.
..\maildir/shackleton-s/all_documents/11399.
..\maildir/shackleton-s/all_documents/114.
..\maildir/shackleton-s/all_documents/1140.
..\maildir/shackleton-s/all_documents/11402.
..\maildir/shackleton-s/all_documents/11408.
..\maildir/shackl

...

..\maildir/germany-c/sent_items/269.
..\maildir/germany-c/sent_items/27.
..\maildir/germany-c/sent_items/270.
..\maildir/germany-c/sent_items/271.
..\maildir/germany-c/sent_items/272.
..\maildir/germany-c/sent_items/273.
..\maildir/germany-c/sent_items/274.
..\maildir/germany-c/sent_items/275.
..\maildir/germany-c/sent_items/276.
..\maildir/germany-c/sent_items/277.
..\maildir/germany-c/sent_items/278.
..\maildir/germany-c/sent_items/279.
..\maildir/germany-c/sent_items/28.
..\maildir/germany-c/sent_items/280.
..\maildir/germany-c/sent_items/281.
..\maildir/germany-c/sent_items/282.
..\maildir/germany-c/sent_items/283.
..\maildir/germany-c/sent_items/284.
..\maildir/germany-c/sent_items/285.
..\maildir/germany-c/sent_items/286.
..\maildir/germany-c/sent_items/287.
..\maildir/germany-c/sent_items/288.
..\maildir/germany-c/sent_items/289.
..\maildir/germany-c/sent_items/29.
..\maildir/germany-c/sent_items/290.
..\maildir/germany-c/sent_items/291.
..\maildir/germany-c/sent_items/292.
..\m

..\maildir/germany-c/sent_items/478.
..\maildir/germany-c/sent_items/479.
..\maildir/germany-c/sent_items/48.
..\maildir/germany-c/sent_items/480.
..\maildir/germany-c/sent_items/481.
..\maildir/germany-c/sent_items/482.
..\maildir/germany-c/sent_items/483.
..\maildir/germany-c/sent_items/484.
..\maildir/germany-c/sent_items/485.
..\maildir/germany-c/sent_items/486.
..\maildir/germany-c/sent_items/488.
..\maildir/germany-c/sent_items/489.
..\maildir/germany-c/sent_items/49.
..\maildir/germany-c/sent_items/490.
..\maildir/germany-c/sent_items/491.
..\maildir/germany-c/sent_items/492.
..\maildir/germany-c/sent_items/493.
..\maildir/germany-c/sent_items/494.
..\maildir/germany-c/sent_items/495.
..\maildir/germany-c/sent_items/496.
..\maildir/germany-c/sent_items/497.
..\maildir/germany-c/sent_items/498.
..\maildir/germany-c/sent_items/499.
..\maildir/germany-c/sent_items/5.
..\maildir/germany-c/sent_items/50.
..\maildir/germany-c/sent_items/501.
..\maildir/germany-c/sent_items/502.
..\mai

..\maildir/germany-c/sent_items/693.
..\maildir/germany-c/sent_items/694.
..\maildir/germany-c/sent_items/695.
..\maildir/germany-c/sent_items/696.
..\maildir/germany-c/sent_items/697.
..\maildir/germany-c/sent_items/698.
..\maildir/germany-c/sent_items/699.
..\maildir/germany-c/sent_items/7.
..\maildir/germany-c/sent_items/70.
..\maildir/germany-c/sent_items/700.
..\maildir/germany-c/sent_items/701.
..\maildir/germany-c/sent_items/702.
..\maildir/germany-c/sent_items/703.
..\maildir/germany-c/sent_items/704.
..\maildir/germany-c/sent_items/705.
..\maildir/germany-c/sent_items/706.
..\maildir/germany-c/sent_items/707.
..\maildir/germany-c/sent_items/708.
..\maildir/germany-c/sent_items/709.
..\maildir/germany-c/sent_items/71.
..\maildir/germany-c/sent_items/710.
..\maildir/germany-c/sent_items/711.
..\maildir/germany-c/sent_items/712.
..\maildir/germany-c/sent_items/713.
..\maildir/germany-c/sent_items/714.
..\maildir/germany-c/sent_items/715.
..\maildir/germany-c/sent_items/716.
..\ma

..\maildir/germany-c/sent_items/927.
..\maildir/germany-c/sent_items/928.
..\maildir/germany-c/sent_items/929.
..\maildir/germany-c/sent_items/93.
..\maildir/germany-c/sent_items/930.
..\maildir/germany-c/sent_items/931.
..\maildir/germany-c/sent_items/932.
..\maildir/germany-c/sent_items/933.
..\maildir/germany-c/sent_items/934.
..\maildir/germany-c/sent_items/935.
..\maildir/germany-c/sent_items/936.
..\maildir/germany-c/sent_items/937.
..\maildir/germany-c/sent_items/938.
..\maildir/germany-c/sent_items/939.
..\maildir/germany-c/sent_items/94.
..\maildir/germany-c/sent_items/940.
..\maildir/germany-c/sent_items/941.
..\maildir/germany-c/sent_items/942.
..\maildir/germany-c/sent_items/943.
..\maildir/germany-c/sent_items/944.
..\maildir/germany-c/sent_items/945.
..\maildir/germany-c/sent_items/946.
..\maildir/germany-c/sent_items/947.
..\maildir/germany-c/sent_items/948.
..\maildir/germany-c/sent_items/949.
..\maildir/germany-c/sent_items/95.
..\maildir/germany-c/sent_items/950.
..\m

Done


```find_signature.py```

In [28]:
#find_signature.py

#!/usr/bin/python

import pickle
import numpy
numpy.random.seed(42)


### The words (features) and authors (labels), already largely processed.
### These files should have been created from the previous (Lesson 10)
### mini-project.
words_file = "your_word_data.pkl"        # CHANGE
authors_file = "your_email_authors.pkl"  # CHANGE
word_data = pickle.load( open(words_file, "r"))
authors = pickle.load( open(authors_file, "r") )


### test_size is the percentage of events assigned to the test set (the
### remainder go into training)
### feature matrices changed to dense representations for compatibility with
### classifier functions in versions 0.15.2 and earlier
from sklearn import cross_validation
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')
features_train = vectorizer.fit_transform(features_train)
features_test  = vectorizer.transform(features_test).toarray()


### a classic way to overfit is to use a small number
### of data points and a large number of features;
### train on only 150 events to put ourselves in this regime
features_train = features_train[:150].toarray()
labels_train   = labels_train[:150]


### your code goes here
from sklearn import tree
from sklearn.metrics import accuracy_score
clf = tree.DecisionTreeClassifier()
clf = clf.fit(features_train,labels_train)
labels_pred = clf.predict(features_test)
acc = accuracy_score(labels_test, labels_pred)
print acc

0.8168373151308305


Gosh, look at how much the accuracy has reduced? Good going :)  

In [29]:
# ok, what is the length of feature importances list
feature_imp_list = clf.feature_importances_
len(feature_imp_list)

37861

In [30]:
# ok quite high, so printing only if above threshold..
import numpy as np
for i,val in np.ndenumerate(feature_imp_list):
    if val > 0.2:
        print i,val

(21323L,) 0.36363636363636365


0.36 looks like a much lesser weight or importance.., so its not an outlier? let us figure out what word it is..

In [31]:
vectorizer.get_feature_names()[21323]

u'houectect'

Remember, we gave only 150 input feature events in find_signature, so it is supposed to over fit (less training data). Because training data is less, resulting in overfit, our accuracy reduced. Thus our goal of overfitting(?!) achieved in this section. 